In [1]:
# DO NOT MODIFY THIS FILE

import random


def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])


In [70]:
# We do Abbey's broken strategy but we look 4 steps back instead of 2

# n will be the number of steps back we look at the opponent's plays, with n = 2 equal to Abbey's strategy
n = 4

def player(prev_play, opponent_history=[],play_order=[{}]):
    if prev_play != "":
        opponent_history.append(prev_play)

    guess = 'R'

    # We gather some data for the first 4 opponent's moves
    if len(opponent_history) > n:
        # The last n opponent's moves
        last_n = ''.join(opponent_history[-n:])

        # We place the last n moves into the play_order dictionary
        if last_n in play_order[0].keys():
            play_order[0][last_n] += 1
        else:
            play_order[0][last_n] = 1

        # We check the next potential plays
        last_n_minus_1 = ''.join(opponent_history[-n+1:])
        potential_plays = [last_n_minus_1 + "R", last_n_minus_1 + "P", last_n_minus_1 + "S"]

        # We find the max out of all of the possible plays in our play_order dictionary
        max = 0
        maxPlay = potential_plays[0]

        for play in potential_plays:
            if play in play_order[0].keys():
                if play_order[0][play] > max:
                    max = play_order[0][play]
                    maxPlay = play

        # We predict the next opponent's play
        prediction = maxPlay[-1]

        # Guess will become whatever beats our predicted value
        ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
        guess = ideal_response[prediction]
        
    return guess


In [71]:
import unittest

class UnitTests(unittest.TestCase):
    print()

    def test_player_vs_quincy(self):
        print("Testing game against quincy...")
        actual = play(player, quincy, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat quincy at least 60% of the time.')

    def test_player_vs_abbey(self):
        print("Testing game against abbey...")
        actual = play(player, abbey, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat abbey at least 60% of the time.')

    def test_player_vs_kris(self):
        print("Testing game against kris...")
        actual = play(player, kris, 1000) >= 60
        self.assertTrue(
            actual, 'Expected player to defeat kris at least 60% of the time.')

    def test_player_vs_mrugesh(self):
        print("Testing game against mrugesh...")
        actual = play(player, mrugesh, 1000) >= 60
        self.assertTrue(
            actual,
            'Expected player to defeat mrugesh at least 60% of the time.')

unittest.main(argv=[''], verbosity=2, exit=False)

test_player_vs_abbey (__main__.UnitTests) ... ok
test_player_vs_kris (__main__.UnitTests) ... ok
test_player_vs_mrugesh (__main__.UnitTests) ... ok
test_player_vs_quincy (__main__.UnitTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK



Testing game against abbey...
Final results: {'p1': 471, 'p2': 255, 'tie': 274}
Player 1 win rate: 64.87603305785123%
Testing game against kris...
Final results: {'p1': 523, 'p2': 166, 'tie': 311}
Player 1 win rate: 75.90711175616835%
Testing game against mrugesh...
Final results: {'p1': 788, 'p2': 176, 'tie': 36}
Player 1 win rate: 81.74273858921161%
Testing game against quincy...
Final results: {'p1': 951, 'p2': 29, 'tie': 20}
Player 1 win rate: 97.0408163265306%
